In [1]:
# importing modules

In [2]:
import pandas as pd
import numpy as np
import vat
from datetime import datetime as dt

In [3]:
# set major variables

In [4]:
# import data

In [5]:
# Read IMPORT file with UNIQUE VAT-IDs to check
# (from IC dispatch sheet of CZ-VAT-Reporting)
xlsx_imp = pd.ExcelFile('test_vatid_xmlrpc_input.xlsx')
df_VAT_IDS = pd.read_excel(xlsx_imp, 'VAT_IDS')

In [6]:
df_VAT_IDS.head(5)

,VAT_ID_OWN,VAT_ID_COUNTERPART
0,DE161045541,ATU14762808
1,DE161045541,ATU14765208
2,DE161045541,ATU19417101
3,DE161045541,ATU19417806
4,DE161045541,ATU19426903


In [7]:
# initiate empty output data frame with retrieved structure
l_keys = ['REQ_DATE', 'VAT_ID_request', 'country_ret', 'VAT_ID_ret', 'valid', 'name', 'address']
df_out = pd.DataFrame(columns = l_keys)
# df_out.head(5)

In [8]:
# MAIN FUNCTIONAL LOOP
# getting responses for ALL VAT-IDs from input file and write VIES-responses
# to df_out data frame

for index, r in df_VAT_IDS.iterrows():
    
    # core request: get response object for counterpart VAT-ID that needs to be confirmed
    o_resp = vat.vies.check_vat(r.VAT_ID_COUNTERPART) # loop index
    
    # core code to transform request response to row that is
    # appended to output data frame df_out

    l_values = o_resp.request_date.strftime("%y_%m_%d").split("§") # .split to transform str directly to list !
    l_values.append(r.VAT_ID_COUNTERPART) # loop index 
    l_values.append(o_resp.country)
    l_values.append(o_resp.vat_number)
    l_values.append(o_resp.valid)
    l_values.append(o_resp.name)
    l_values.append(o_resp.address)

    # zipping key and value lists to dictionary and append as new row to df_out
    d_resp = dict(zip(l_keys, l_values))
    # print(d_response)
    df_out = df_out.append(d_resp, ignore_index = True)


In [9]:
df_out.head(5)

,REQ_DATE,VAT_ID_request,country_ret,VAT_ID_ret,valid,name,address
0,19_12_11,ATU14762808,AT,U14762808,True,MEDIA MARKT TV-Hifi-Elektro Wien XI Gesellscha...,Landwehrstraße 6\nAT-1110 Wien
1,19_12_11,ATU14765208,AT,U14765208,True,MEDIA MARKT TV-Hifi-Elektro Wien XXI Gesellsch...,Brünner Straße 72A\nAT-1210 Wien
2,19_12_11,ATU19417101,AT,U19417101,True,Media - Saturn Beteiligungsges.m.b.H.,SCS-Buerocenter/B 2\nAT-2334 Vösendorf
3,19_12_11,ATU19417806,AT,U19417806,True,MEDIA MARKT TV-Hifi-Elektro Gesellschaft m.b.H.,SCS Nordring\nAT-2334 Vösendorf
4,19_12_11,ATU19426903,AT,U19426903,True,MEDIA MARKT TV-Hifi-Elektro Gesellschaft m.b.H.,SC Seiersberg 9/T 6/3\nAT-8054 Seiersberg


In [10]:
len(df_out)

512

In [11]:
writer = pd.ExcelWriter('_VIES_VAT_ID_verification.xlsx', engine='xlsxwriter')
df_out.to_excel(writer, sheet_name='VIES_responses', engine='xlsxwriter')
writer.save()